In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib

type_and_name_by_index = {}


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

/home/v_stepanov/anaconda3/envs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train

,region,company_type,company_name,inn,incorporation_year,okved_osn_code,init_equities,init_equity_types,purpose,other_reasons_for_check,check_date,type,kpp,type_nasel_punkt,name_nasel_punkt,index,risk_category,zipcode
0,adygea,юридическое лицо,муниципальное бюджетное дошкольное образовател...,105034720,1996.0,88.0,NaN,NaN,проверка соблюдения трудового законодательства,NaN,2018-07-15,выездная,1.0,город,майкоп,385000.0,5,385009.0
1,adygea,юридическое лицо,государственное казенное учреждение здравоохра...,105018083,1999.0,86.0,NaN,NaN,проверка соблюдения трудового законодательства,NaN,2018-11-15,выездная,1.0,город,майкоп,385017.0,5,385009.0
2,adygea,юридическое лицо,государственное бюджетное учреждение здравоохр...,105020893,1995.0,86.0,NaN,NaN,проверка соблюдения трудового законодательства,NaN,2018-02-15,выездная,1.0,город,майкоп,385018.0,5,385009.0
3,adygea,юридическое лицо,государственное бюджетное учреждение здравоохр...,105016209,1996.0,86.0,NaN,NaN,проверка соблюдения трудового законодательства,NaN,2018-02-15,выездная,1.0,город,майкоп,385000.0,5,385009.0
4,adygea,юридическое лицо,государственное бюджетное учреждение республик...,105035498,2001.0,88.0,NaN,NaN,проверка соблюдения трудового законодательства,NaN,2018-02-15,выездная,1.0,город,майкоп,385000.0,5,385009.0
5,adygea,юридическое лицо,"открытое акционерное общество ""зарем""",105000819,1992.0,28.0,6.479800e+04,уставный капитал,права потребителей,NaN,2018-06-15,выездная,1.0,город,майкоп,385001.0,3,385009.0
6,adygea,юридическое лицо,"общество с ограниченной ответственностью ""югор...",2303023723,2005.0,8.0,1.000000e+04,уставный капитал,права потребителей,NaN,2018-03-15,выездная,1.0,город,майкоп,385000.0,4,385009.0
7,adygea,юридическое лицо,федеральное государственное бюджетное образова...,105014177,1994.0,85.0,NaN,NaN,права потребителей,NaN,2018-12-15,выездная,1.0,город,майкоп,385000.0,4,385009.0
8,adygea,юридическое лицо,государственное бюджетное учреждение здравоохр...,105020251,1997.0,86.0,NaN,NaN,права потребителей,NaN,2018-03-15,выездная,1.0,город,майкоп,385006.0,4,385009.0
9,adygea,юридическое лицо,"общество с ограниченной ответственностью ""мпк""...",107011090,2006.0,11.0,1.000000e+04,уставный капитал,права потребителей,NaN,2018-03-15,выездная,1.0,город,майкоп,385000.0,5,385009.0


In [4]:

type_by_short = {
    "г": "город",
    "дп": "дачный поселок",
    "дп": "дачный поселок",
    "кп": "курортный поселок",
    "пгт": "поселок городского типа",
    "п/о": "почтовое отделение",
    "рп": "рабочий поселок",
    "аул": "аул",
    "тер": "территория",
    "ст-ца": "станица",
    "с": "село"
}

def get_city_and_type_by_index(index):
    if index in type_and_name_by_index:
        return type_and_name_by_index[index]
    if str(index).startswith('19'):
        return ('город', 'санкт-петербург')
    if str(index).startswith('10') or str(index).startswith('11') or str(index).startswith('12'):
        return ('город', 'москва')
    headers = { 'User-Agent' : 'Mozilla/5.0' }
    req = urllib.request.Request(url="https://gdeposylka.ru/courier/russian-post/{}".format(index), headers=headers)
    try:
        page = urllib.request.urlopen(req)
    except Exception as e:
        type_and_name_by_index[index] = (None, None)
        return (None, None)
        
    soup = BeautifulSoup(page, 'html.parser')
    addr = soup.find("address").text
    for token in addr.strip().split(','):
        token = token.strip()
        
        for key in type_by_short:
            if token.startswith(key):
                t = key
                name = token[len(key) + 1:].lower()
                type_and_name_by_index[index] = (type_by_short[t], name)
                return type_by_short[t], name
    type_and_name_by_index[index] = (None, None)
    return (None, None)


In [5]:
get_city_and_type_by_index(191015)

('город', 'санкт-петербург')

In [6]:
train.type_nasel_punkt.unique()

array(['город', 'аул', 'хутор', 'поселок', 'станица',
       'поселок городского типа', 'село', nan, 'промышленная зона',
       'городок', 'железнодорожная станция', 'рабочий поселок', 'станция',
       'микрорайон', 'аал', 'населенный пункт', 'деревня',
       'ж/д останов. (обгонный) пункт', 'поселок и(при) станция(и)',
       'почтовое отделение', 'километр', 'автодорога',
       'поселок сельского типа', 'местечко', 'курортный поселок',
       'городской поселок', 'территория', 'слобода', 'лесничество',
       'садовое неком-е товарищество', 'жилой район'], dtype=object)

In [7]:
for x, y in zip(*np.unique(train[train.type_nasel_punkt.notnull()].type_nasel_punkt.values, return_counts=True)):
    print(x, y)

аал 51
автодорога 26
аул 161
город 32371
городок 4
городской поселок 1
деревня 1204
ж/д останов. (обгонный) пункт 4
железнодорожная станция 76
жилой район 1
километр 2
курортный поселок 1
лесничество 2
местечко 9
микрорайон 1
населенный пункт 9
поселок 4187
поселок городского типа 1108
поселок и(при) станция(и) 26
поселок сельского типа 1
почтовое отделение 10
промышленная зона 10
рабочий поселок 1187
садовое неком-е товарищество 29
село 13846
слобода 1
станица 1992
станция 35
территория 14
хутор 482


In [8]:
len(train[train.name_nasel_punkt.isnull()])



12115

In [9]:
cnt = 0
def transform_row(row):
    global cnt

    if np.isnan(float(row['zipcode'])):
        return row
    t, city = get_city_and_type_by_index(int(float(row['zipcode'])))
    if city is not None and t is not None:
        row.name_nasel_punkt = city
        row.type_nasel_punkt = t
    cnt += 1
    if cnt % 100 == 0:
        print(cnt)
    return row


In [10]:
train = train.apply(transform_row, axis=1)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [11]:
train

,region,company_type,company_name,inn,incorporation_year,okved_osn_code,init_equities,init_equity_types,purpose,other_reasons_for_check,check_date,type,kpp,type_nasel_punkt,name_nasel_punkt,index,risk_category,zipcode
0,adygea,юридическое лицо,муниципальное бюджетное дошкольное образовател...,105034720,1996.0,88.0,NaN,NaN,проверка соблюдения трудового законодательства,NaN,2018-07-15,выездная,1.0,город,майкоп,385000.0,5,385009.0
1,adygea,юридическое лицо,государственное казенное учреждение здравоохра...,105018083,1999.0,86.0,NaN,NaN,проверка соблюдения трудового законодательства,NaN,2018-11-15,выездная,1.0,город,майкоп,385017.0,5,385009.0
2,adygea,юридическое лицо,государственное бюджетное учреждение здравоохр...,105020893,1995.0,86.0,NaN,NaN,проверка соблюдения трудового законодательства,NaN,2018-02-15,выездная,1.0,город,майкоп,385018.0,5,385009.0
3,adygea,юридическое лицо,государственное бюджетное учреждение здравоохр...,105016209,1996.0,86.0,NaN,NaN,проверка соблюдения трудового законодательства,NaN,2018-02-15,выездная,1.0,город,майкоп,385000.0,5,385009.0
4,adygea,юридическое лицо,государственное бюджетное учреждение республик...,105035498,2001.0,88.0,NaN,NaN,проверка соблюдения трудового законодательства,NaN,2018-02-15,выездная,1.0,город,майкоп,385000.0,5,385009.0
5,adygea,юридическое лицо,"открытое акционерное общество ""зарем""",105000819,1992.0,28.0,6.479800e+04,уставный капитал,права потребителей,NaN,2018-06-15,выездная,1.0,город,майкоп,385001.0,3,385009.0
6,adygea,юридическое лицо,"общество с ограниченной ответственностью ""югор...",2303023723,2005.0,8.0,1.000000e+04,уставный капитал,права потребителей,NaN,2018-03-15,выездная,1.0,город,майкоп,385000.0,4,385009.0
7,adygea,юридическое лицо,федеральное государственное бюджетное образова...,105014177,1994.0,85.0,NaN,NaN,права потребителей,NaN,2018-12-15,выездная,1.0,город,майкоп,385000.0,4,385009.0
8,adygea,юридическое лицо,государственное бюджетное учреждение здравоохр...,105020251,1997.0,86.0,NaN,NaN,права потребителей,NaN,2018-03-15,выездная,1.0,город,майкоп,385006.0,4,385009.0
9,adygea,юридическое лицо,"общество с ограниченной ответственностью ""мпк""...",107011090,2006.0,11.0,1.000000e+04,уставный капитал,права потребителей,NaN,2018-03-15,выездная,1.0,город,майкоп,385000.0,5,385009.0


In [14]:
len(type_and_name_by_index)

680

In [15]:
test = test.apply(transform_row, axis=1)

63500
63600
63700
63800
63900
64000
64100
64200
64300
64400
64500
64600
64700
64800
64900
65000
65100
65200
65300
65400
65500
65600
65700
65800
65900
66000
66100
66200
66300
66400
66500
66600
66700
66800
66900
67000
67100
67200
67300
67400
67500
67600
67700
67800
67900
68000
68100
68200
68300
68400
68500
68600
68700
68800
68900
69000
69100
69200
69300
69400
69500
69600
69700
69800
69900
70000
70100
70200
70300
70400
70500


In [ ]:
train.to